# Классификация комментариев

У интернет-магазина «Викишоп» новый сервис - пользователи могут редактировать и дополнять описания товаров. Клиенты предлагают свои правки и комментируют изменения других. 


Необходим построить модель, которая будет классифицировать комментарии на позитивные и негативные (определять токсичность),а затем отправлять их на модерацию. 

В распоряжении набор данных с разметкой о токсичности правок.

**Требование:**
* Метрика качества *F1* >= 0.75. 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Выгрузка-данных" data-toc-modified-id="Выгрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Выгрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Анализ-результатов-моделей" data-toc-modified-id="Анализ-результатов-моделей-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Анализ результатов моделей</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Проверка-модели" data-toc-modified-id="Проверка-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели</a></span><ul class="toc-item"><li><span><a href="#Проверка-модели-на-тестовых-данных" data-toc-modified-id="Проверка-модели-на-тестовых-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Проверка модели на тестовых данных</a></span></li><li><span><a href="#Проверка-на-адекватность" data-toc-modified-id="Проверка-на-адекватность-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Проверка на адекватность</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Заключение" data-toc-modified-id="Заключение-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Заключение</a></span></li></ul></div>

In [1]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
import nltk 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer 
import re 
import matplotlib.pyplot as plt 
import seaborn as sns 
from time import time
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
nltk.download('averaged_perceptron_tagger')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Выгрузка данных

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [4]:
# посмотрим на основную информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
# проверим на пропуски
df.isna().sum()

text     0
toxic    0
dtype: int64

In [7]:
# также проверим на дубликаты
df.duplicated().sum()

0

In [8]:
# проверим, нет ли каких-то аномальных значений в столбце с целевым признаком
df['toxic'].unique()

array([0, 1])

In [9]:
# посмотрим на соотношение классов
print('Соотношения классов:')
print(df['toxic'].value_counts(normalize=True))

Соотношения классов:
0    0.898388
1    0.101612
Name: toxic, dtype: float64


### Вывод:
- Данные были выгружены
- Типы данных соответствуют заявленных
- Дубликатов и пропусков не обнаружено
- Наблюдается дисбаланс классов

## Подготовка данных

In [10]:
RANDOM_STATE = 74

In [11]:
lemmatizer = WordNetLemmatizer()

In [12]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def preproc_text(text):
    text = text.lower()
    lemm_text = " ".join(lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

In [13]:
# применим функцию к нашим данным
df['lemm_text'] = df['text'].apply(preproc_text)

In [14]:
# разделим выборки на фичи и таргет
X = df.drop(['toxic', 'text'], axis = 1)
y = df['toxic']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=RANDOM_STATE,  
                                                    stratify=y)

In [16]:
# удалим стоп-слова и векторизуем строки
nltk.download('stopwords')
stopwords = list(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
X_train = count_tf_idf.fit_transform(X_train['lemm_text'])
X_test = count_tf_idf.transform(X_test['lemm_text'])

### Вывод:
Произвелась подготовка данных:
- Был устранен дисбаланс классов
- Произведена лемматизация строк, а также выделение стоп-слов
- данные были разделены на тренировочную и тестовую выборки

## Обучение

### LogisticRegression

In [19]:
# объявим модель и гиперпараметры
log_model = LogisticRegression()

param_lr = [{'C':[1,5,10],
             'class_weight':['balanced']}]

In [20]:
# применим cv 
gs_lr = GridSearchCV(log_model, 
                     param_lr, 
                     cv=3, 
                     scoring='f1') 

In [21]:
# проведем кросс-валидацию для текста без лемматизации и засекем время обучения
start = time()

gs_lr.fit(X_train, y_train)

end = time()

lr_time = (end-start)/60 

In [22]:
# замеряем F1 на  кросс-валидации
score_log_model = (gs_lr.best_score_)

In [23]:
print("F1 для LogisticRegression модели:", round(score_log_model,2))

F1 для LogisticRegression модели: 0.76


### RandomForestClassifier

In [25]:
# инициализируем модель
rf_model = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced_subsample') 

# укажем гиперпараметры для перебора
param_rf = {'n_estimators': [50, 100],
            'max_depth': [10, 12],
            'criterion':['gini', 'entropy', 'log_loss']}

In [26]:
# применим cv без лемматизации
gs_rf = GridSearchCV(
             rf_model, 
             param_rf, 
             cv=3, 
             scoring='f1',
             n_jobs=-1) 

In [27]:
# обучим модель на данных без лемматизации и засекем время
start = time()

gs_rf.fit(X_train, y_train)

end = time()

rf_time = (end-start)/60 

In [28]:
# замеряем F1 на  кросс-валидации
score_rf_model = (gs_rf.best_score_)

In [29]:
print("F1 для RandomForestClassifier модели:", round((score_rf_model), 2))

F1 для RandomForestClassifier модели: 0.38


### CatBoostClassifier

In [30]:
# инициализируем модель
cat_model = CatBoostClassifier(random_state=RANDOM_STATE, verbose=False) 

# укажем гиперпараметры для перебора
param_cat = {'iterations':[100],
             'depth':[10],
             'learning_rate':[0.1]}

In [31]:
# применим cv без лемматизации
gs_cat = GridSearchCV(
              cat_model, 
              param_cat, 
              cv=3, 
              scoring='f1',
              n_jobs=-1)

In [32]:
# обучим модель на данных без лемматизации и засекем время
start = time()

gs_cat.fit(X_train, y_train)

end = time()

cat_time = (end-start)/60 

In [33]:
# замеряем F1 на  кросс-валидации
score_cat_model = (gs_cat.best_score_)

In [34]:
print("F1 для CatBoostClassifier модели:", round((score_cat_model), 2))

F1 для CatBoostClassifier модели: 0.68


### Анализ результатов моделей

In [35]:
# оформляем результаты в таблицу
res = [[round((score_log_model), 2), lr_time],
       [round((score_rf_model), 2), rf_time],
       [round((score_cat_model), 2), cat_time]]

model = ["LogisticRegression", "RandomForestClassifier", "CatBoostClassifier"]

In [36]:
results = pd.DataFrame(data=res, index=model, columns=[ "F1_score", 'fit_time'])

results

,F1_score,fit_time
LogisticRegression,0.76,3.266108
RandomForestClassifier,0.38,12.067446
CatBoostClassifier,0.68,109.005903


### Вывод:
- Были обучены три модели классификации.
- Лучше всего справилась модель LogisticRegression.

## Проверка модели

### Проверка модели на тестовых данных

In [38]:
# теперь проверим модель на тестовых данных
pred_lr = gs_lr.best_estimator_.predict(X_test)
print(f'F1 на тестовой выборке для LogisticRegression: {round(f1_score(y_test, pred_lr), 2)}')

F1 на тестовой выборке для LogisticRegression: 0.76


- Условие выполено: метрика f1 > 0.75

### Проверка на адекватность

In [39]:
# инициируем модель, предсказывающий константное значение
dummy_model = DummyClassifier(strategy='stratified')

In [40]:
# обучим модель и сделаем предсказание
dummy_model.fit(X_train, y_train)

pred_dummy = dummy_model.predict(X_test)

score_dummy_model = f1_score(y_test, pred_dummy)

print("F1 для Dummy модели:", score_dummy_model)

F1 для Dummy модели: 0.1016949152542373


- Dummy модель с поставленной задачей не справилась
- Проверка на адекватность пройдена

### Вывод:
- Проверили модель на тестовых данных 
- Метрика F1 больше ожидаемой
- Модель прошла тест на адекватность

## Заключение

В ходе выполненй работы были произведены:
- Выгрузка и подготовка данных
- Обучение моделей и поиск лучшей
- Проверка модели на тестовой выборке, а также проверка модели на адекватность 
- Лучшей моделью оказалась LogisticRegression